In [1]:
import tensorflow as tf
import os
import tarfile
import requests

In [2]:
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

inception_pretrain_model_dir = "inception_model"
if not os.path.exists(inception_pretrain_model_dir):
    os.makedirs(inception_pretrain_model_dir)

filename = inception_pretrain_model_url.split('/')[-1]
filepath = os.path.join(inception_pretrain_model_dir,filename)

if not os.path.exists(filepath):
    print('download:',filename)
    #如果将stream=True 则会推迟响应内容的下载
    r = requests.get(inception_pretrain_model_url, stream=True)
    with open(filepath, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
print('finish: ',filename)
#解压文件
#参数'r:gz'的意思，采用gzip格式解压并打开文件
tarfile.open(filepath, 'r:gz').extractall(inception_pretrain_model_dir)

log_dir = 'logs/inception_log'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# classfy_image_graph_def.pb 为google训练好的模型
inception_graph_def_fiel = os.path.join(inception_pretrain_model_dir,'classify_image_graph_def.pb')
with tf.Session() as sess:
    #创建一个图来存放google训练好的模型
    with tf.gfile.FastGFile(inception_graph_def_fiel, 'rb') as f:
        graph_def = tf.GraphDef()
#         graph_def.ParseFromSring(f.read())
        graph_def.ParseFromString(f.read())
    # 解释 https://blog.csdn.net/qq_34106574/article/details/82686389
    #这个函数提供了一种方法来导入序列化的TensorFlow GraphDef协议缓冲区，并将GraphDef中的各个对象提取为tf。
    #操作对象。一旦提取出来，这些对象就会被放到当前的默认图形中。。参考tf.Graph.as_graph_def 来创建 a GraphDef proto.
        tf.import_graph_def(graph_def, name='')
    
    # 保存图的结构
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    writer.close()



finish:  inception-2015-12-05.tgz


In [3]:
print(filename)
print(filepath)

inception-2015-12-05.tgz
inception_model\inception-2015-12-05.tgz
